- [문제 링크](https://www.acmicpc.net/problem/1774)

### 풀이
- 이미 주어진 간선이 있으면 어떻게 할 것인가?를 묻는 문제임
1. Kruskal : 해당 간선들을 먼저 Union하고, 값은 더하지 않음
2. Prim : 해당 간선들의 가중치를 0으로 함

In [15]:
# 1. Kruskal

import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline
sys.setrecursionlimit(int(1e6))

def find(x):
    if parent[x] != x:
        parent[x] = find(parent[x])
    return parent[x]

def union(a, b):
    if a == b:
        return
    
    a = find(a)
    b = find(b)
    
    if a > b:
        a, b  = b, a
    
    parent[b] = a
    
def dist(a, b):
    dist = ((loc[b][0] - loc[a][0])**2 + 
            (loc[b][1] - loc[a][1])**2 )**(1/2)
    return dist


N, M = map(int, input().split())

# 1. 좌표를 얻고 점들을 구성하는 모든 간선의 가중치를 구함
loc = [(-1, -1)] * (N + 1)
edges = []
for i in range(1, N + 1):
    loc[i] = tuple(map(int, input().split()))
    for j in range(1, i):
        w = dist(i, j)
        edges.append((w, i, j))
        
edges.sort()

# 2. 가장 작은 점들부터 union을 시도함
parent = [i for i in range(N + 1)] # 1 ~ N만 사용
min_span = 0

# 이미 연결된 간선부터 union해준다
for _ in range(M):
    a, b = map(int, input().split())
    if find(a) == find(b):
        continue
    union(a, b)

# 이미 연결되었다면 if 조건으로 걸러짐
for w, a, b in edges:
    if find(a) == find(b):
        continue
    
    union(a, b)
    min_span += w

print("{:.2f}".format(min_span))

4.00


- 모든 간선을 구하지 않고 문제를 푸는 방법은 없겠지?

In [60]:
# 2. Prim
# 예제 잘 풀리고 다르게 값을 줘도 잘 풀리는데 뭐가 문제인지 모르겠다.
import sys
import heapq
import math
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

N, M = map(int, input().split())
graph = [[(-1, -1, -1)] * (N + 1) for _ in range(N + 1)]

loc = [(-1, -1)] * (N + 1)

# 이미 연결된 점 처리하기

# 1. 각 점의 좌표를 구함
for i in range(1, N + 1):
    a, b = map(int, input().split())
    loc[i] = (a, b)

    
# 2. 그래프 그리기 : 이미 연결된 점들은 거리를 0으로 넣는다
graph = [[[-1, -1, -1]] * (N + 1) for _ in range(N + 1)]

def dist(a, b):
    dist = math.sqrt(((loc[b][0] - loc[a][0])** 2 +
           (loc[b][1] - loc[a][1]) ** 2))
    return dist

for i in range(1, N + 1):
    for j in range(i, N + 1):
        graph[i][j] = [dist(i, j), i, j]
        graph[j][i] = [dist(i, j), j, i]

# 3. 이미 연결된 값들은 길이를 0으로 갱신함
for _ in range(M):
    a, b = map(int, input().split())
    graph[a][b][0] = 0
    graph[b][a][0] = 0
    
def prim(start_node):
    visited = [0] * (N + 1)
    hp = []
    visited[start_node] = 1
    length = 0
    
    for w, a, b in graph[start_node]:
        if w != -1:
            heapq.heappush(hp, (w, a, b))

    while hp:
        w, _, now = heapq.heappop(hp)
        if visited[now] == 0:
            visited[now] = 1
            length += w
            for next_w, now, next_ in graph[now]:
                if visited[next_] == 0:
                    heapq.heappush(hp, (next_w, now, next_))
            
    return length


ans = prim(1)
print("{:.2f}".format(ans))

1.41


### Prim으로 풀다가 든 궁금증
- 이런 상황을 가정해보자
    - 4개의 점이 있고, 1을 중심으로 2, 3, 4가 연결됐을 때가 최소
    - 1, 2가 이미 연결된 상황
---
- Prim은 방문 리스트를 이용한다고 생각하고, 이미 연결됐으니까 1, 2를 방문 리스트에 방문했다고 찍어버리면 `3-1-4`를 연결할 수 없음. `1`이 이미 방문한적이 있기 때문에 `heapq`에 들어가는 요소로 고려되지 않기 때문에
- 찾아보니까, **이미 연결된 값들은 거리를 0으로 해서 그래프에 넣으면 된다고 한다.**

In [61]:
import math
import heapq
import sys
input = sys.stdin.readline
N, M = map(int, input().split())


def prim(start):
    result = 0
    visited = [False] * (N+1)
    q = []
    heapq.heappush(q, (0, start))
    while q:
        dist, node = heapq.heappop(q)
        if visited[node]:
            continue
        visited[node] = True
        result += dist
        for ndist, nnode in arr[node]:
            heapq.heappush(q, (ndist, nnode))
    return result


# 노드 위치 정보
nodes = [[]]
for _ in range(N):
    nodes.append(list(map(int, input().split())))

# 이미 연결되어 있는 값을 저장
connected = set()
for _ in range(M):
    x, y = map(int, input().split())
    connected.add((x, y))
    connected.add((y, x))

# 연결리스트
arr = [[] for _ in range(N+1)]
for i in range(1, N+1):
    for j in range(i+1, N+1):
        # 이미 연결되어 있다면 dist를 0으로
        if (i, j) in connected:
            arr[i].append((0, j))
            arr[j].append((0, i))
            continue
        # 연결 되지 않았다면
        dist = math.sqrt((nodes[i][0]-nodes[j][0])**2 +
                         (nodes[i][1]-nodes[j][1])**2)
        arr[i].append((dist, j))
        arr[j].append((dist, i))

print("%.2f" % prim(1))

ValueError: not enough values to unpack (expected 2, got 0)